# Debugging, profiling, and optimization

**Naomi Ceder, @naomiceder**

- **Chair, Python Software Foundation**
- **Quick Python Book, 3rd ed**
- **Dick Blick Art Materials**

**This notebook will be available for a week at https://github.com/nceder/training/tree/master/bloomberg**

**Online sign in - `ATND <GO>`, code YNMSJS - from email**

**or email rbasil@bloomberg.net**


Debugging, profiling, and optimization - Covers basic debugging strategies from logging to using pdb to disassembling, as well as profilig to find bottlenecks. Optimization strategies and C-extensions in both Python 2 and 3. Cython, PyPy, etc

```
Monday
- AM: Intermediate Python
- PM: Iterators, Generators, Collections

Tuesday
- AM: Pythonic Coding
- PM: Moving to Python 3

Wednesday
- AM: Data cleaning
- PM: Intermediate Python (repeat)

Thursday
- AM: Moving to Python 3 (repeat)
- PM: Debugging Profiling Timing

Friday
- AM: Code organization and packaging
- PM: Pythonic coding (repeat)
```

## Course Assumptions

* My course outline is only a general guide
* We can be guided by your needs/interests
* I need direction on what those are
* The more we interact the better the outcome is likely to be

### You

* What do you do?
* What coding experience do you have?
* What are your repetitive hassles and time sinks?
* What problems do you want/hope to solve with code?

## What we'll do

* Discuss debugging strategies
  * print statements
  * pdb
* Profiling
  * timing
  * disassembly
* Optimization
  * strategies
  * command line
  * code 
  * C extensions
  * Faster Pythons


## Python 3 vs Python 2
### Friends don't let friends use Python 2

I'm going to be using Python 3, but if you're using Python 2, you should be okay if you do the following:
1. `from __future__ import print_function` - `print` becomes a function, requires ()
2. `from __future__ import division` - `/` is no longer integer division and will return a `real`
3. use "new style" classes, i.e., `class my_class(object)`


Other key differences will be pointed out as they arise.


## Virtual environments

Before we get started, it's a good idea to use virtual environments.

* create separate environments for different interpreters, e.g., Python 2.7, Python 3.5, Python 3.6
* keep dependencies, library versions, etc separate
* part of Python 3

```
$ python3 -m venv my_project 
```
or at Bloomberg
```
$ virtualenv -p python3 my_project$ source my_project/bin/activate
$ pip install <packages>
... work on stuff...
$ deactivate
```
Save packages for an environment:

    $ pip freeze > requirements.txt

Install saved requirements:

    $ pip install -r requirements.txt

## Debugging
* Code runs, but doesn't behave as expected *or*
* Code runs, then crashes
* Some state is set incorrectly
* Or... the path of execution isn't what is expected


<img src="debug_print1.png">
<img src="debug_print2.png">

### print statements

* arguably the oldest and simplest debugging approach
* easy to dump pretty much anything, anywhere 
* enduring popularity


### Drawbacks of print statements

* you need to be able see STDOUT
* can be overwhelmed in loops, etc
* need to be (manually) inserted in code and removed

### Reason to use print statements

* fast & flexible
* good for exploring a problem fast

### logging

* Similar to print statements - exposing state and execution path
* logging library for standard library

#### Key differences

* levels - debug, info, warning, error, critical
* likely to be left in place
* various channels
* different outputs (even by level)
* more overhead to set up

Logging Levels

```
NOTSET     0
DEBUG     10
INFO      20
WARNING   30
ERROR     40
CRITICAL  50
```

Log messages at or above the logger's log level will be logged. So, NOTSET means that **all** messages will be logged, while WARNING would mean that warnings, errors, and criticals would be logged.


In [34]:
import logging
import os

# create the logger
logger = logging.getLogger('my_process')
logger.setLevel(logging.DEBUG)


# set up file for debug level messages
os.remove('process.log')
file_handler = logging.FileHandler('process.log')
file_handler.setLevel(logging.DEBUG)
logger.addHandler(file_handler)

# setup console for errors only
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.ERROR) 
logger.addHandler(console_handler)

logger.debug("Debug - This goes only to the file")
logger.info("Info - This goes only to the file")
logger.warning("Warning - This goes only to the file")
 
logger.error("Error - This goes to the console and the file")
logger.critical(" C ritical - This goes to the console and the file")

Error - This goes to the console and the file
Critical - This goes to the console and the file


In [35]:
print(open('process.log').read())

Debug - This goes only to the file
Info - This goes only to the file
Warning - This goes only to the file
Error - This goes to the console and the file
Critical - This goes to the console and the file



In [ ]:
# fibonacci with recursion

def fib_recursive(number):
    result = 0
    if number in [1, 2]:
        result = 1
    else:
        try:
            result =  fib_recursive(number - 1) + fib_recursive(number - 2)
        except Exception as e:
            logger.error("{} {} {}".format(e, number, result))
            
    logger.debug("{} {}".format(number, result))  # only to file
    return result
  
fib_recursive(-6)  

maximum recursion depth exceeded in comparison -2973 0


In [1]:
print(open('process.log').read()) 

Debug - This goes only to the file
Info - This goes only to the file
Warning - This goes only to the file
Error - This goes to the console and the file
Critical - This goes to the console and the file
unsupported operand type(s) for -: 'str' and 'int' 6 0
6 0
2 1
1 1
3 2
2 1
4 3
2 1
1 1
3 2
5 5
2 1
1 1
3 2
2 1
4 3
6 8
maximum recursion depth exceeded in comparison -2973 0
-2973 0



### Debugging with Exceptions

* Trapping exception and logging/printing 
* Using existing exception (with message parameter)
* Create custom exception


### Downsides of Exceptions

* you need to know where to trap them - correct size of blocks for try...except
* recovery is often tricky
* more effort than than `print` statements

### (faulthandler in Python 3)

Library that enables dumping minimal traceback even on segfaults and lockups. (see docs)



### Debug flags

* Global variable
* `if` statement
* effect much like logging levels



In [5]:
# fibonacci with recursion
DEBUG = True

def fib_recursive(number):
    if number in [1, 2]:
        result = 1
    else:
        result =  fib_recursive(number - 1) + fib_recursive(number - 2)
    
    if DEBUG:
        print(number, result)
    return result

fib_recursive(6)

2 1
1 1
3 2
2 1
4 3
2 1
1 1
3 2
5 5
2 1
1 1
3 2
2 1
4 3
6 8


8

In [6]:
import dis
dis.dis(fib_recursive)

  5           0 LOAD_FAST                0 (number)
              2 LOAD_CONST               1 ((1, 2))
              4 COMPARE_OP               6 (in)
              6 POP_JUMP_IF_FALSE       14

  6           8 LOAD_CONST               2 (1)
             10 STORE_FAST               1 (result)
             12 JUMP_FORWARD            24 (to 38)

  8     >>   14 LOAD_GLOBAL              0 (fib_recursive)
             16 LOAD_FAST                0 (number)
             18 LOAD_CONST               2 (1)
             20 BINARY_SUBTRACT
             22 CALL_FUNCTION            1
             24 LOAD_GLOBAL              0 (fib_recursive)
             26 LOAD_FAST                0 (number)
             28 LOAD_CONST               3 (2)
             30 BINARY_SUBTRACT
             32 CALL_FUNCTION            1
             34 BINARY_ADD
             36 STORE_FAST               1 (result)

 10     >>   38 LOAD_GLOBAL              1 (DEBUG)
             40 POP_JUMP_IF_FALSE       52

 11         

### ` __debug__` keyword

* `__debug__` is a constant set when the interpreter starts

* `__debug__` is true if -O is **not** set on the commandline when the interpreter is run

### Without -O switch

In [8]:
print(__debug__)
__debug__ = False

SyntaxError: assignment to keyword (<ipython-input-8-f6598caa108a>, line 2)

In [9]:
# fibonacci with recursion

def fib_recursive(number):
    if number in [1, 2]:
        result = 1
    else:
        result =  fib_recursive(number - 1) + fib_recursive(number - 2)
    
    if __debug__:
        print(number, result)
    return result

fib_recursive(6) 

2 1
1 1
3 2
2 1
4 3
2 1
1 1
3 2
5 5
2 1
1 1
3 2
2 1
4 3
6 8


8

In [10]:
import dis
dis.dis(fib_recursive)

  4           0 LOAD_FAST                0 (number)
              2 LOAD_CONST               1 ((1, 2))
              4 COMPARE_OP               6 (in)
              6 POP_JUMP_IF_FALSE       14

  5           8 LOAD_CONST               2 (1)
             10 STORE_FAST               1 (result)
             12 JUMP_FORWARD            24 (to 38)

  7     >>   14 LOAD_GLOBAL              0 (fib_recursive)
             16 LOAD_FAST                0 (number)
             18 LOAD_CONST               2 (1)
             20 BINARY_SUBTRACT
             22 CALL_FUNCTION            1
             24 LOAD_GLOBAL              0 (fib_recursive)
             26 LOAD_FAST                0 (number)
             28 LOAD_CONST               3 (2)
             30 BINARY_SUBTRACT
             32 CALL_FUNCTION            1
             34 BINARY_ADD
             36 STORE_FAST               1 (result)

 10     >>   38 LOAD_GLOBAL              1 (print)
             40 LOAD_FAST                0 (number)
    

### (switch to 3_O kernel)

In [1]:
print(__debug__)

False


In [2]:
# fibonacci with recursion

def fib_recursive(number):
    if number in [1, 2]:
        result = 1
    else:
        result =  fib_recursive(number - 1) + fib_recursive(number - 2)
    
    if __debug__:
        print(number, result)
    return result

fib_recursive(6)

8

In [3]:
import dis
dis.dis(fib_recursive)

  4           0 LOAD_FAST                0 (number)
              2 LOAD_CONST               1 ((1, 2))
              4 COMPARE_OP               6 (in)
              6 POP_JUMP_IF_FALSE       14

  5           8 LOAD_CONST               2 (1)
             10 STORE_FAST               1 (result)
             12 JUMP_FORWARD            24 (to 38)

  7     >>   14 LOAD_GLOBAL              0 (fib_recursive)
             16 LOAD_FAST                0 (number)
             18 LOAD_CONST               2 (1)
             20 BINARY_SUBTRACT
             22 CALL_FUNCTION            1
             24 LOAD_GLOBAL              0 (fib_recursive)
             26 LOAD_FAST                0 (number)
             28 LOAD_CONST               3 (2)
             30 BINARY_SUBTRACT
             32 CALL_FUNCTION            1
             34 BINARY_ADD
             36 STORE_FAST               1 (result)

 11     >>   38 LOAD_FAST                1 (result)
             40 RETURN_VALUE


### assert statements

* Convenient way to use assertions in code
* Formatat: `assert <expression>`
* `assert` will raise AssertionError exception if expression is not True

#### `assert` with one expression

`assert <expression>`

is equivalent to:

```
if __debug__:
    if not <expression>:
        raise AssertionError
```

### (switch to 3 kernel)

In [1]:
# fibonacci with recursion

def fib_recursive(number):

    assert number > 0
    
    if number in [1, 2]:
        result = 1
    else:
        result =  fib_recursive(number - 1) + fib_recursive(number - 2)
    
    return result

fib_recursive(-6)

AssertionError: 

In [2]:
import dis
dis.dis(fib_recursive)

  5           0 LOAD_FAST                0 (number)
              2 LOAD_CONST               1 (0)
              4 COMPARE_OP               4 (>)
              6 POP_JUMP_IF_TRUE        12
              8 LOAD_GLOBAL              0 (AssertionError)
             10 RAISE_VARARGS            1

  7     >>   12 LOAD_FAST                0 (number)
             14 LOAD_CONST               2 ((1, 2))
             16 COMPARE_OP               6 (in)
             18 POP_JUMP_IF_FALSE       26

  8          20 LOAD_CONST               3 (1)
             22 STORE_FAST               1 (result)
             24 JUMP_FORWARD            24 (to 50)

 10     >>   26 LOAD_GLOBAL              1 (fib_recursive)
             28 LOAD_FAST                0 (number)
             30 LOAD_CONST               3 (1)
             32 BINARY_SUBTRACT
             34 CALL_FUNCTION            1
             36 LOAD_GLOBAL              1 (fib_recursive)
             38 LOAD_FAST                0 (number)
             40 

### (switch to 3_O kernel)

In [1]:
# WITH -O flag

# fibonacci with recursion

def fib_recursive(number):

    assert number > 0
    
    if number in [1, 2]:
        result = 1
    else:
        result =  fib_recursive(number - 1) + fib_recursive(number - 2)
    
    return result

fib_recursive(-6)

RecursionError: maximum recursion depth exceeded in comparison

In [2]:
import dis
dis.dis(fib_recursive)
 

  9           0 LOAD_FAST                0 (number)
              2 LOAD_CONST               1 ((1, 2))
              4 COMPARE_OP               6 (in)
              6 POP_JUMP_IF_FALSE       14

 10           8 LOAD_CONST               2 (1)
             10 STORE_FAST               1 (result)
             12 JUMP_FORWARD            24 (to 38)

 12     >>   14 LOAD_GLOBAL              0 (fib_recursive)
             16 LOAD_FAST                0 (number)
             18 LOAD_CONST               2 (1)
             20 BINARY_SUBTRACT
             22 CALL_FUNCTION            1
             24 LOAD_GLOBAL              0 (fib_recursive)
             26 LOAD_FAST                0 (number)
             28 LOAD_CONST               3 (2)
             30 BINARY_SUBTRACT
             32 CALL_FUNCTION            1
             34 BINARY_ADD
             36 STORE_FAST               1 (result)

 14     >>   38 LOAD_FAST                1 (result)
             40 RETURN_VALUE


#### `assert` with two expressions

`assert <expression_1>, <expression_2>`

is equivalent to:

```
if __debug__:
    if not <expression_1>:
        raise AssertionError(<expression_2>)
```

### (switch to 3 kernel)

In [1]:
# fibonacci with recursion

def fib_recursive(number):

    assert number > 0, f"{number} is not > 0" 
    result = 1
    result =  fib_recursive(number - 1) + fib_recursive(number - 2)
    return result

fib_recursive(-6)

AssertionError: -6 is not > 0

In [2]:
import dis
dis.dis(fib_recursive) 

  5           0 LOAD_FAST                0 (number)
              2 LOAD_CONST               1 (0)
              4 COMPARE_OP               4 (>)
              6 POP_JUMP_IF_TRUE        22
              8 LOAD_GLOBAL              0 (AssertionError)
             10 LOAD_FAST                0 (number)
             12 FORMAT_VALUE             0
             14 LOAD_CONST               2 (' is not > 0')
             16 BUILD_STRING             2
             18 CALL_FUNCTION            1
             20 RAISE_VARARGS            1

  6     >>   22 LOAD_CONST               3 (1)
             24 STORE_FAST               1 (result)

  7          26 LOAD_GLOBAL              1 (fib_recursive)
             28 LOAD_FAST                0 (number)
             30 LOAD_CONST               3 (1)
             32 BINARY_SUBTRACT
             34 CALL_FUNCTION            1
             36 LOAD_GLOBAL              1 (fib_recursive)
             38 LOAD_FAST                0 (number)
             40 LOAD_CO

### pdb (Python Debugger)

* interactive source debugger
* break points, single stepping, evaluation, etc
* post-mortem
* programmatic debugging

### Debugging a running program

`import pdb; pdb.set_trace()`

### `breakpoint()` 

`breakpoint(*args, **kws)`

This function drops you into the debugger at the call site. Specifically, it calls `sys.breakpointhook()`, 
passing args and kws straight through. By default, `sys.breakpointhook()` calls `pdb.set_trace()` 
expecting no arguments. In this case, it is purely a convenience function so you don’t have to 
explicitly import pdb or type as much code to enter the debugger. However, `sys.breakpointhook()` 
can be set to some other function and `breakpoint()` will automatically call that, allowing you 
to drop into the debugger of choice.

**New in version 3.7**

### Debugging a script

`python -m pdb myscript.py`

or

```
import pdb
import test_module
pdb.run(test_module.my_funct())
```


### Debugging a function

```
import pdb
import test_module
pdb.runcall(test_module.my_funct, 2)
```


#### pdb commands

```
c - continue
s - step
n - next
a - arguments
b - breakpoints
cl - clear breakpoints
w - stack trace
p - print expression
pp - pretty print expression
l, ll - list code
```

(and many others - see documentation)

In [3]:
# fibonacci with recursion

def fib_recursive(number):

    if number in [1, 2]:
        result = 1
    else:
        result =  fib_recursive(number - 1) + fib_recursive(number - 2)
    
    import pdb; pdb.set_trace() 
    # breakpoint()
    return result



fib_recursive(3 )

> <ipython-input-3-6a68ab00cd75>(12)fib_recursive()
-> return result
(Pdb) n
--Return--
> <ipython-input-3-6a68ab00cd75>(12)fib_recursive()->1
-> return result
(Pdb) p number
2
(Pdb) n
--Call--
> <ipython-input-3-6a68ab00cd75>(3)fib_recursive()
-> def fib_recursive(number):
(Pdb) p number
1
(Pdb) n
> <ipython-input-3-6a68ab00cd75>(5)fib_recursive()
-> if number in [1, 2]:
(Pdb) ll 
  3  	def fib_recursive(number):
  4  	
  5  ->	    if number in [1, 2]:
  6  	        result = 1
  7  	    else:
  8  	        result =  fib_recursive(number - 1) + fib_recursive(number - 2)
  9  	
 10  	    import pdb; pdb.set_trace()
 11  	    # breakpoint()
 12  	    return result
(Pdb) n
> <ipython-input-3-6a68ab00cd75>(6)fib_recursive()
-> result = 1
(Pdb) n
> <ipython-input-3-6a68ab00cd75>(10)fib_recursive()
-> import pdb; pdb.set_trace()
(Pdb) c
> <ipython-input-3-6a68ab00cd75>(12)fib_recursive()
-> return result
(Pdb) c
> <ipython-input-3-6a68ab00cd75>(12)fib_recursive()
-> return result
(Pdb) c


2

## Step through your favorite script

Take a few minutes and add and use the `import pdb; pdb.set_trace()` to step through some code. 

Optionally, use the dis module to disassemble a small function. 

## Profiling

Getting a picture of how the code uses resources

### Performance testing code sections

### tools

#### profile and trace

* cProfile
* trace

In [4]:
def fib_recursive(number):

    if number  in [1, 2]:
        result = 1
    else:
        result =  fib_recursive(number - 1) + fib_recursive(number - 2)
    
    return result


In [5]:
import cProfile
cProfile.run('fib_recursive(25)')


         150052 function calls (4 primitive calls) in 0.058 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 150049/1    0.058    0.000    0.058    0.058 <ipython-input-4-6d54bdd8d386>:1(fib_recursive)
        1    0.000    0.000    0.058    0.058 <string>:1(<module>)
        1    0.000    0.000    0.058    0.058 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [6]:
import re
cProfile.run('re.compile("foo|bar")')


         214 function calls (207 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        2    0.000    0.000    0.000    0.000 enum.py:284(__call__)
        2    0.000    0.000    0.000    0.000 enum.py:526(__new__)
        1    0.000    0.000    0.000    0.000 enum.py:836(__and__)
        1    0.000    0.000    0.000    0.000 re.py:232(compile)
        1    0.000    0.000    0.000    0.000 re.py:271(_compile)
        1    0.000    0.000    0.000    0.000 sre_compile.py:249(_compile_charset)
        1    0.000    0.000    0.000    0.000 sre_compile.py:276(_optimize_charset)
        2    0.000    0.000    0.000    0.000 sre_compile.py:453(_get_iscased)
        1    0.000    0.000    0.000    0.000 sre_compile.py:461(_get_literal_prefix)
        1    0.000    0.000    0.000    0.000 sre_compile.py:492(_get_charset_prefix)
        1   

In [7]:
# trace

import sys
import trace

# create a Trace object, telling it what to ignore, and whether to
# do tracing or line-counting or both.
tracer = trace.Trace(
    ignoredirs=[sys.prefix, sys.exec_prefix],
    trace=1,
    count=1)

tracer.run('re.compile("foo|bar")')

# make a report, placing output in the current directory
r = tracer.results()
r.write_results(show_missing=True, coverdir=".")

 --- modulename: re, funcname: compile
re.py(234):     return _compile(pattern, flags)
 --- modulename: re, funcname: _compile
re.py(273):     if isinstance(flags, RegexFlag):
re.py(275):     try:
re.py(276):         return _cache[type(pattern), pattern, flags]


In [8]:
print(open('re.cover').read())

       #
       # Secret Labs' Regular Expression Engine
       #
       # re-compatible interface for the sre matching engine
       #
       # Copyright (c) 1998-2001 by Secret Labs AB.  All rights reserved.
       #
       # This version of the SRE library can be redistributed under CNRI's
       # Python 1.6 license.  For any other use, please contact Secret Labs
       # AB (info@pythonware.com).
       #
       # Portions of this engine have been developed in cooperation with
       # CNRI.  Hewlett-Packard provided funding for 1.6 integration and
       # other compatibility work.
       #
       
       r"""Support for regular expressions (RE).
       
       This module provides regular expression matching operations similar to
       those found in Perl.  It supports both 8-bit and Unicode strings; both
       the pattern and the strings being processed can contain null bytes and
       characters outside the US ASCII range.
       
       Regular expressions can contain both

## tracemalloc

Trace memory allocated, compare snapshots of memory

In [9]:
import tracemalloc

tracemalloc.start()

x = "-".join(str(n) for n in range(100))

snapshot = tracemalloc.take_snapshot()
top_stats = snapshot.statistics('lineno', cumulative=True)

print("[ Top 10 ]")
for stat in top_stats[:10]:
    print(stat)


[ Top 10 ]
<ipython-input-9-923502c79f2f>:5: size=2546 B, count=2, average=1273 B
/usr/lib/python3.7/codeop.py:133: size=621 B, count=8, average=78 B
<ipython-input-9-923502c79f2f>:7: size=432 B, count=1, average=432 B
/home/naomi/.virtualenvs/jupyter3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2899: size=104 B, count=3, average=35 B


## Try it out

Run cProfile or trace on some of your code, and see if the results make sense.

## Timing

#### timers

* timeit.timeit()
* time.clock()/time.time()

#### timeit module

* commandline  - `python -m timeit "expression"`
* callable interface - `import timeit`
* Timer class

#### timeit commandline

`python -m timeit -n <number> -r <repetitions of timer> -s <setup expression> <expression to time>`

In [10]:
"-".join(str(n) for n in range(100))

'0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99'

In [ ]:
def join_digits_100():
    result = "-".join(str(n) for n in range(100))
    return result

In [11]:
# Note that from commandline 10000 runs and 3 repetitions is used by default

$ python -m timeit '"-".join(str(n) for n in range(100))'
100000 loops, best of 3: 18.2 usec per loop
$ python -m timeit '"-".join([str(n) for n in range(100)])'
100000 loops, best of 3: 16.5 usec per loop
$ python -m timeit '"-".join(map(str, range(100)))'
100000 loops, best of 3: 10.8 usec per loop


SyntaxError: invalid syntax (<ipython-input-11-1b3a48f743c0>, line 3)

In [12]:
 def concat_str(n):
    result = ""
    for x in range(n-1):
        result += str(x) + "-"
    result += str(n-1)
    return result

concat_str(100)

'0-1-2-3-4-5-6-7-8-9-10-11-12-13-14-15-16-17-18-19-20-21-22-23-24-25-26-27-28-29-30-31-32-33-34-35-36-37-38-39-40-41-42-43-44-45-46-47-48-49-50-51-52-53-54-55-56-57-58-59-60-61-62-63-64-65-66-67-68-69-70-71-72-73-74-75-76-77-78-79-80-81-82-83-84-85-86-87-88-89-90-91-92-93-94-95-96-97-98-99'

In [13]:
import timeit

# using timeit(), the number of runs needs to be specified or 1000000 is used. Can't repeat
a = timeit.timeit('"-".join(str(n) for n in range(100000))', number=100)
print(a)

b = timeit.timeit('"-".join([str(n) for n in range(100000)])', number=100)
print(b)

c = timeit.timeit('"-".join(map(str, range(100000)))', number=100)
print(c)

d = timeit.timeit('concat_str(100000)', setup="from __main__ import concat_str", 
                number=100)
print(d)



6.260520209994866
6.392004443987389
5.901348232990131
8.630485610003234


In [ ]:
import timeit
 
# using repeat, the number of runs needs to be specified or 1000000 is used. repeats defaults to 3
a = timeit.repeat('"-".join(str(n) for n in range(100))', number=10000, repeat=3)
print(a)

b = timeit.repeat('"-".join([str(n) for n in range(100)])', number=10000, repeat=3)
print(b)

c = timeit.repeat('"-".join(map(str, range(100)))', number=10000, repeat=3)
print(c)

d = timeit.repeat('concat_str(100)', setup="from __main__ import concat_str", number=10000, repeat=3)
print(d)

%timeit concat_str(100 )


## Experiment with timeit

Run time it on some various snippets of code to get a feel for how it works in your environment.

### Py-spy

https://github.com/benfred/py-spy

## Optimization

“The real problem is that programmers have spent far too much time worrying about efficiency in the wrong places and at the wrong times; **premature optimization is the root of all evil (or at least most of it) in programming.**” 

-- Donald Knuth

## Code optimizations

### Dictionaries

### strings

* `.join()` over concatenation

In [14]:


def test_concat():
    new_string = ""
    for x in range(1000000):
        new_string += "test"
        
def test_join():
    data = []
    for x in range(1000000):
        data.append("test")

    new_string = "".join(data)


In [15]:
%timeit test_concat()

%timeit test_join()



348 ms ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
176 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### list operations

(deques if needed)

In [16]:
from collections import deque

a_list = []
a_deque = deque()

def test_list_queue():
    for x in range(10000):
        a_list.insert(x, 0)
    
    for x in range(10000):
        a_list.pop(0)

def test_deque():
    for x in range(10000):
        a_deque.appendleft(x)
    
    for x in range(10000):
        a_deque.popleft()

 

In [17]:
%timeit test_list_queue()

%timeit test_deque()

12.5 ms ± 78.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.22 ms ± 85.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Optimizing for memory

* Python 3 over Python 2 - generators and other lazy tricks
* Encouraging garbage collection - reference count, scope, etc
* -OO to removed built doc strings
* see MicroPython for ideas

## Optimizing for speed

* Code optimization - loops, etc
* Caching
* External libraries

## caching

* functools - lrucache


In [18]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib_recursive_cache(number):
    if number in [1, 2]:
        result = 1
    else:
        result =  fib_recursive_cache(number - 1) + fib_recursive_cache(number - 2)
    
    return result
    


In [19]:
%timeit fib_recursive(10) 
%timeit fib_recursive_cache(10)

10.3 µs ± 75.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
54.4 ns ± 0.749 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


 <img src="JakeVP_lru.png">

## Code in C

### Cython

`$ pip install cython`

### (Switch to 3.6 kernel)


In [ ]:
!pip install cython


In [6]:

%load_ext Cython

In [7]:
def fib(number):
    if number < 2:
        return number
    return fib(number-1)+fib(number-2) 

In [8]:
%%cython
def fib_cython(number):
    if number < 2:
        return number
    return fib_cython(number-1)+fib_cython(number-2)

In [9]:
%%cython
cpdef long fib_cython_typed(long number):
    if number < 2:
        return number
    return fib_cython_typed(number-1)+fib_cython_typed(number-2)

In [10]:
# fibonacci via looping

def fib_loop(number):
    cur_fib,next_fib = 1,1
    for i in range(number - 1):
        cur_fib,  next_fib = next_fib, cur_fib + next_fib
    return cur_fib




In [11]:
%%cython

# fibonacci via looping, cython, typed


cpdef long fib_loop_cython_typed(long number):
    cdef long cur_fib, next_fib
    cur_fib, next_fib = 1,1
    for i in range(number - 1):
        cur_fib,next_fib = next_fib,cur_fib+next_fib
    return cur_fib



In [12]:
%timeit fib(10)

%timeit fib_cython(10)
 
%timeit fib_cython_typed(10)

%timeit fib_loop(10)

%timeit fib_loop_cython_typed(10)

20.9 µs ± 790 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
6.53 µs ± 279 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
203 ns ± 12.1 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
607 ns ± 4.86 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
57 ns ± 7.36 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


File fib.pyx
```
def fib(number):
    if number < 2:
        return number
    return fib(number-1)+fib(number-2)
```
File setup.py
```
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("fib.pyx")
    )
```
Build with:
`$ python setup.py build_ext --inplace`

In [1]:
import fib 

In [2]:
%timeit fib.fib(10)

6.87 µs ± 98 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


### ctypes

* provides C compatible data types and interface
* allows using DLL's and shared libraries
* can be used to wrap functions in those libraries

In [3]:
from ctypes import *
cdll.LoadLibrary("libc.so.6") 
libc = CDLL("libc.so.6")
libc.printf("%s\n", "hello") 

-1

In [13]:
print(libc.time(None))
import time
print(time.time())

1561681634
1561681634.8309903


**C function fib_rec**
```
int fib_rec(int number){
  if (number < 2) {
    return number;
  }
  return fib_rec(number-1)+fib_rec(number-2);
}
```

**C function fib_loop**
```
int fib_rec(int number){
  if (number < 2) {
    return number;
  }
  return fib_rec(number-1)+fib_rec(number-2);
}
```
**Compile to share lib `libfib.so`**
```
$ cc -fPIC -shared -o libfib.so fib_c.c
```

In [14]:
import ctypes

ctypes_fib = ctypes.CDLL('/home/naomi/Documents/bloomberg/libfib.so')



In [15]:
%timeit ctypes_fib.fib_loop(10) 

305 ns ± 11.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [16]:
%timeit ctypes_fib.fib_rec(10)

%timeit ctypes_fib.fib_loop(10)

773 ns ± 43.2 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
298 ns ± 3.76 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### CFFI

* intended to keep as much in Python as possible
* API - uses C compiler to create an interface library
* ABI - accesses libraries directly, like C-Types


In [17]:
from cffi import FFI

ffi = FFI()

lib = ffi.dlopen('/home/naomi/Documents/bloomberg/libfib.so')
print('Loaded lib {0}'.format(lib))

ffi.cdef("""
int fib_rec(int number);
int fib_loop(int number);
""")
lib.fib_rec(10)

Loaded lib <cffi.api._make_ffi_library.<locals>.FFILibrary object at 0x7f203d0f3b38>


55

In [18]:
%timeit lib.fib_rec(10)

%timeit lib.fib_loop(10)

730 ns ± 54.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
254 ns ± 5.19 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### C extensions

* Adding new built-in objects to Python
* Interface to C libraries
* Uses C-Python API (Python.h)
* Not as portable, and not recommended

### Other options

* [Pybind11](https://github.com/pybind/pybind11) - Used at Bloomberg for C++ libraries
* [SWIG](http://www.swig.org/)
* [Boost.Python](http://www.boost.org/doc/libs/1_59_0/libs/python/doc/)

## Faster Pythons

### Jython

* Python on Java VM
* No GIL, so multiple threads
* Java libraries

### Jython drawbacks

* need to install Java VM
* only 2.7
* slow load time (load VM, then interpreter)

### PyPy

* Python in Python (R-Python, anyway)
* Very fast


### PyPy drawbacks

* not very mature
* Very complex and difficult to build/configure



## Resources

### Debugging

* [Python Logging HOWTO](https://docs.python.org/3/howto/logging.html)
* [Python Logging Cookbook](https://docs.python.org/3/howto/logging-cookbook.html)
* [Effortless Logging (Mario Corchero, Bloomberg)](https://www.youtube.com/watch?v=Pbz1fo7KlGg)
* [Using Assertions Effectively](https://wiki.python.org/moin/UsingAssertionsEffectively)
* [faulthandler module docs](https://docs.python.org/3/library/faulthandler.html)
* [pdb module docs](https://docs.python.org/3/library/pdb.html)
* [inspect module docs](https://docs.python.org/3/library/inspect.html)

### Profiling

* [Python Profilers](https://docs.python.org/3/library/profile.html)
* [timeit module](https://docs.python.org/3/library/timeit.html)
* [time module](https://docs.python.org/3/library/time.html)
* [Py-Spy](https://github.com/benfred/py-spy)

### Optimizing

* [Python Speed](https://wiki.python.org/moin/PythonSpeed)
* [Python Speed Performance Tips](https://wiki.python.org/moin/PythonSpeed/PerformanceTips)
* [Cython](http://cython.org/)
* [ctypes module](https://docs.python.org/3/library/ctypes.html)
* [C Extensions](https://docs.python.org/3/extending/extending.html)
* [CFFI](https://cffi.readthedocs.io/en/latest/overview.html#)
* [Building C Extensions with distutils](https://docs.python.org/3/extending/building.html)
* [Jython](http://www.jython.org/)
* [PyPy](https://pypy.org/)

